# Despliegue de un Modelo de Clasificación como Servicio Web con FastAPI

Este notebook documenta el proceso completo para desplegar un modelo de clasificación de vinos como un servicio web utilizando FastAPI. Aprenderemos cómo entrenar un modelo de machine learning y exponerlo a través de una API REST.

## Contenido

1. [Preparación del Entorno](#1-preparación-del-entorno)
2. [Carga y Entrenamiento del Modelo](#2-carga-y-entrenamiento-del-modelo)
3. [Implementación de la API con FastAPI](#3-implementación-de-la-api-con-fastapi)
4. [Configuración del Sistema de Logging](#4-configuración-del-sistema-de-logging)
5. [Ejecución del Servicio](#5-ejecución-del-servicio)
6. [Pruebas y Ejemplos de Uso](#6-pruebas-y-ejemplos-de-uso)
7. [Documentación Automática](#7-documentación-automática)
8. [Conclusiones](#8-conclusiones)

## 1. Preparación del Entorno

Primero, necesitamos instalar las dependencias necesarias para nuestro proyecto. Estas incluyen FastAPI para crear la API, Uvicorn como servidor ASGI, scikit-learn para el modelo de machine learning, y otras bibliotecas de soporte.

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.68.1 uvicorn==0.15.0 scikit-learn==0.24.2 pandas==1.3.3 numpy==1.21.2 python-multipart==0.0.5 loguru==0.5.3

Creamos un archivo `requirements.txt` para documentar las dependencias del proyecto:

In [ ]:
%%writefile requirements.txt
fastapi==0.68.1
uvicorn==0.15.0
scikit-learn==0.24.2
pandas==1.3.3
numpy==1.21.2
python-multipart==0.0.5
loguru==0.5.3

## 2. Carga y Entrenamiento del Modelo

Para este ejemplo, utilizaremos el dataset de vinos de scikit-learn. Cargaremos los datos, los dividiremos en conjuntos de entrenamiento y prueba, y entrenaremos un modelo de Random Forest para la clasificación.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Cargar el dataset de vinos
wine = load_wine()
X, y = wine.data, wine.target

# Explorar las características del dataset
print(f"Características del dataset: {wine.feature_names}")
print(f"Clases: {wine.target_names}")
print(f"Dimensiones de X: {X.shape}")

# Crear un DataFrame para visualizar mejor los datos
df = pd.DataFrame(X, columns=wine.feature_names)
df['target'] = y
df.head()

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
print(f"Precisión del modelo: {accuracy_score(y_test, y_pred):.4f}")
print("Informe de clasificación:
")
print(classification_report(y_test, y_pred, target_names=wine.target_names))

## 3. Implementación de la API con FastAPI

Ahora que tenemos un modelo entrenado, vamos a implementar una API REST utilizando FastAPI para exponer nuestro modelo como un servicio web. Crearemos un archivo `main.py` con la implementación de la API.

In [ ]:
%%writefile main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from loguru import logger
import numpy as np

# Configuración del logger
logger.add("api.log", rotation="500 MB")

app = FastAPI(
    title="API de Clasificación de Vinos",
    description="API para predecir la clase de vino usando el dataset de sklearn",
    version="1.0.0"
)

# Cargar y preparar los datos
X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

class WineFeatures(BaseModel):
    alcohol: float
    malic_acid: float
    ash: float
    alcalinity_of_ash: float
    magnesium: float
    total_phenols: float
    flavanoids: float
    nonflavanoid_phenols: float
    proanthocyanins: float
    color_intensity: float
    hue: float
    od280_od315_of_diluted_wines: float
    proline: float

@app.post("/predict")
async def predict_wine(features: WineFeatures):
    try:
        logger.info(f"Recibiendo predicción para características: {features}")
        
        # Convertir características a array
        X = np.array([
            features.alcohol, features.malic_acid, features.ash, features.alcalinity_of_ash,
            features.magnesium, features.total_phenols, features.flavanoids, features.nonflavanoid_phenols,
            features.proanthocyanins, features.color_intensity, features.hue, 
            features.od280_od315_of_diluted_wines, features.proline
        ]).reshape(1, -1)
        
        # Realizar predicción
        prediction = model.predict(X)[0]
        probabilities = model.predict_proba(X)[0]
        
        # Mapear clases a tipos de vino
        wine_types = {0: "Clase 0", 1: "Clase 1", 2: "Clase 2"}
        
        result = {
            "prediction": int(prediction),
            "probabilities": {f"clase_{i}": float(prob) for i, prob in enumerate(probabilities)},
            "message": f"Vino clasificado como: {wine_types[prediction]}"
        }
        
        logger.info(f"Predicción exitosa: {result}")
        return result
        
    except Exception as e:
        logger.error(f"Error durante la predicción: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    return {"status": "ok"}

### Explicación del Código

En el código anterior:  

1. **Configuración de FastAPI**: Creamos una instancia de FastAPI con metadatos como título, descripción y versión.

2. **Carga y Entrenamiento del Modelo**: Cargamos el dataset de vinos, dividimos los datos y entrenamos un modelo de Random Forest.

3. **Definición del Modelo de Datos**: Utilizamos Pydantic para definir la estructura de datos esperada en las solicitudes a la API.

4. **Endpoint de Predicción**: Implementamos un endpoint POST `/predict` que recibe las características del vino y devuelve la predicción del modelo.

5. **Endpoint de Salud**: Implementamos un endpoint GET `/health` para verificar el estado de la API.

## 4. Configuración del Sistema de Logging

Hemos configurado un sistema de logging utilizando la biblioteca Loguru. Esto nos permite registrar información importante sobre las solicitudes y respuestas de la API, así como cualquier error que pueda ocurrir durante la ejecución.

```python
# Configuración del logger
logger.add("api.log", rotation="500 MB")
```

Los logs se guardan en el archivo `api.log` y se rotan cuando alcanzan un tamaño de 500 MB. Esto es útil para monitorear el comportamiento de la API en producción y para depurar problemas.

## 5. Ejecución del Servicio

Para ejecutar el servicio, utilizamos Uvicorn, un servidor ASGI de alto rendimiento. En un entorno de producción, ejecutaríamos el siguiente comando:

In [ ]:
# En un entorno real, ejecutaríamos este comando en la terminal
!python -m uvicorn main:app --reload

Este comando inicia el servidor Uvicorn con la opción `--reload`, que reinicia automáticamente el servidor cuando se detectan cambios en el código. Esto es útil durante el desarrollo, pero en producción se recomienda omitir esta opción.

Una vez que el servidor está en ejecución, la API estará disponible en `http://localhost:8000`.

## 6. Pruebas y Ejemplos de Uso

Ahora que tenemos nuestra API en funcionamiento, vamos a realizar algunas pruebas para verificar que todo funciona correctamente. Utilizaremos la biblioteca `requests` para enviar solicitudes a la API.

In [ ]:
import requests
import json
import matplotlib.pyplot as plt

# URL de la API (asumiendo que está ejecutándose localmente)
API_URL = "http://localhost:8000"

# Verificar el estado de la API
response = requests.get(f"{API_URL}/health")
print(f"Estado de la API: {response.json()}")

# Datos de ejemplo para la predicción (tomados del conjunto de prueba)
sample_index = 0  # Índice de la muestra que queremos usar
sample_features = X_test[sample_index]

# Crear el payload para la solicitud
payload = {
    "alcohol": float(sample_features[0]),
    "malic_acid": float(sample_features[1]),
    "ash": float(sample_features[2]),
    "alcalinity_of_ash": float(sample_features[3]),
    "magnesium": float(sample_features[4]),
    "total_phenols": float(sample_features[5]),
    "flavanoids": float(sample_features[6]),
    "nonflavanoid_phenols": float(sample_features[7]),
    "proanthocyanins": float(sample_features[8]),
    "color_intensity": float(sample_features[9]),
    "hue": float(sample_features[10]),
    "od280_od315_of_diluted_wines": float(sample_features[11]),
    "proline": float(sample_features[12])
}

print("Payload de la solicitud:")
print(json.dumps(payload, indent=4))

# Realizar la solicitud de predicción
response = requests.post(f"{API_URL}/predict", json=payload)

# Verificar la respuesta
if response.status_code == 200:
    result = response.json()
    print("
Respuesta de la API:")
    print(json.dumps(result, indent=4))
    
    # Visualizar las probabilidades
    probabilities = result["probabilities"]
    classes = list(probabilities.keys())
    values = list(probabilities.values())
    
    plt.figure(figsize=(10, 6))
    plt.bar(classes, values, color='skyblue')
    plt.xlabel('Clase')
    plt.ylabel('Probabilidad')
    plt.title('Probabilidades de Predicción')
    plt.ylim(0, 1)
    for i, v in enumerate(values):
        plt.text(i, v + 0.02, f'{v:.2f}', ha='center')
    plt.show()
    
    # Comparar con la etiqueta real
    true_label = y_test[sample_index]
    predicted_label = result["prediction"]
    print(f"
Etiqueta real: {true_label} ({wine.target_names[true_label]})
          f"Etiqueta predicha: {predicted_label} ({wine.target_names[predicted_label]})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

### Ejemplo de Payload para la API

A continuación se muestra un ejemplo de payload que podemos enviar a la API para obtener una predicción:

In [ ]:
example_payload = {
    "alcohol": 13.0,
    "malic_acid": 1.5,
    "ash": 2.3,
    "alcalinity_of_ash": 15.0,
    "magnesium": 100.0,
    "total_phenols": 2.5,
    "flavanoids": 3.0,
    "nonflavanoid_phenols": 0.3,
    "proanthocyanins": 1.5,
    "color_intensity": 5.0,
    "hue": 1.0,
    "od280_od315_of_diluted_wines": 3.0,
    "proline": 1000.0
}

print(json.dumps(example_payload, indent=4))

## 7. Documentación Automática

Una de las ventajas de FastAPI es que genera automáticamente documentación interactiva para nuestra API. Esta documentación está disponible en dos formatos:

1. **Swagger UI**: Disponible en `http://localhost:8000/docs`
2. **ReDoc**: Disponible en `http://localhost:8000/redoc`

A continuación se muestra una captura de pantalla de la documentación generada por Swagger UI:

![Swagger UI](https://fastapi.tiangolo.com/img/index/index-01-swagger-ui-simple.png)

Esta documentación interactiva permite a los usuarios de la API entender cómo utilizarla, qué parámetros espera cada endpoint y qué respuestas devuelve. También permite probar la API directamente desde el navegador.

## 8. Conclusiones

En este notebook, hemos aprendido cómo desplegar un modelo de clasificación como un servicio web utilizando FastAPI. Hemos cubierto los siguientes aspectos:

1. **Preparación del Entorno**: Instalación de dependencias y configuración del proyecto.

2. **Carga y Entrenamiento del Modelo**: Utilización del dataset de vinos de scikit-learn para entrenar un modelo de Random Forest.

3. **Implementación de la API**: Creación de endpoints para predicción y verificación de salud.

4. **Configuración del Sistema de Logging**: Utilización de Loguru para registrar información importante.

5. **Ejecución del Servicio**: Uso de Uvicorn como servidor ASGI.

6. **Pruebas y Ejemplos de Uso**: Envío de solicitudes a la API y visualización de resultados.

7. **Documentación Automática**: Aprovechamiento de la documentación generada por FastAPI.

Este enfoque nos permite exponer modelos de machine learning como servicios web, lo que facilita su integración con otras aplicaciones y sistemas. Además, FastAPI nos proporciona herramientas para garantizar la calidad y la seguridad de nuestra API, como validación de datos, documentación automática y manejo de errores.

### Recursos Adicionales

- [Documentación oficial de FastAPI](https://fastapi.tiangolo.com/)
- [Documentación de scikit-learn](https://scikit-learn.org/stable/documentation.html)
- [Documentación de Uvicorn](https://www.uvicorn.org/)
- [Documentación de Loguru](https://loguru.readthedocs.io/en/stable/index.html)